# Оценка работы RAG с помошью библиотеки ragas

### Setup:

1. В этом руководстве для выполнения некоторых метрик используется OpenAI, поэтому убедитесь, что ваш ключ OpenAI готов и доступен в вашей среде.
2. Вставте Ваш ключ для OpenAI в вайл 'env.txt'
3. Установите нужные библиотеки. Рекомендуется использовать изолированое пространство (virtual environment).
    * pip install -r requirements.txt
4. запускайте блокнот



In [1]:
import os
from dotenv import dotenv_values

# Load environment variables from env.txt file
env_vars = dotenv_values('env.txt')

# Explicitly set the environment variables from the env.txt file to override the global variables if set
for key, value in env_vars.items():
    os.environ[key] = value

# Verify the environment variables are set correctly
if 'OPENAI_API_KEY' in os.environ:
    print(f"OpenAI API Key successfully loaded")
else:
    print("Failed to load OpenAI API Key")

OpenAI API Key successfully loaded


# Данные

Создаём набор данных в нужном формате с нужными названиями столбцов

In [2]:
from datasets import Dataset

# NOTE: ragas documentation is not up to date (14.05.2024). The requred Dataset structure is:
# Dataset.from_dict(
#         {
#             'question': list[str],
#             'contexts': list[list[str]],
#             'ground_truth': list[str],
#             'answer': list[str]
#         }
#     )

questions = [
    "чем представлены органические остатки?", 
    "что найдено в кремнистых сланцах железорудной формации Канады?",
]
contexts = [
    ["В протерозойских отложениях органические остатки встречаются намного чаще, чем в архейских. Они представлены известковыми выделениями сине-зелёных водорослей, ходами червей, остатками кишечнополостных. Кроме известковых водорослей, к числу древнейших растительных остатков относятся скопления графито-углистого вещества, образовавшегося в результате разложения Corycium enigmaticum. В кремнистых сланцах железорудной формации Канады найдены нитевидные водоросли, грибные нити и формы, близкие современным кокколитофоридам. В железистых кварцитах Северной Америки и Сибири обнаружены железистые продукты жизнедеятельности бактерий."],
    ["В протерозойских отложениях органические остатки встречаются намного чаще, чем в архейских. Они представлены известковыми выделениями сине-зелёных водорослей, ходами червей, остатками кишечнополостных. Кроме известковых водорослей, к числу древнейших растительных остатков относятся скопления графито-углистого вещества, образовавшегося в результате разложения Corycium enigmaticum. В кремнистых сланцах железорудной формации Канады найдены нитевидные водоросли, грибные нити и формы, близкие современным кокколитофоридам. В железистых кварцитах Северной Америки и Сибири обнаружены железистые продукты жизнедеятельности бактерий."],
]
# expected answers
ground_truths = [
    "известковыми выделениями сине-зелёных водорослей",
    "нитевидные водоросли, грибные нити",
]
# answers from your LLM
answers = [
    "Органические остатки могут быть представлены различными способами, включая растения, животные, микроорганизмы, гумус и другие органические материалы.",
    "В кремнистых сланцах железорудной формации Канады обычно находятся богатые железом рудные отложения.",
]

In [3]:
my_dataset = Dataset.from_dict(
    {
        'question': questions,
        'contexts': contexts,
        'ground_truth': ground_truths,
        'answer': answers,
    }
)

# Метрики

Ragas предоставляет несколько метрик для оценки различных аспектов ваших систем RAG:

1. Ретривер: Предлагает **context_precision** и **context_recall**, которые измеряют производительность вашей системы извлечения.

2. Генератор (LLM): Предоставляет метрику **faithfulness**, которая измеряет галлюцинации, и **answer_relevancy**, которая измеряет, насколько ответы соответствуют вопросу.

В Ragas доступно множество других метрик, ознакомьтесь с руководством по метрикам, чтобы узнать больше.

Теперь давайте импортируем эти метрики и разберемся, что они обозначают.


In [4]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

Здесь мы используем четыре метрики, но что они собой представляют?

1. **Faithfulness (достоверность)** - измеряет фактическое соответствие ответа контексту, основанному на вопросе.

2. **Context_precision (точность контекста)** - измеряет, насколько релевантен извлеченный контекст вопросу, передавая качество поискового конвейера.

3. **Answer_relevancy  (релевантность ответа)** - измеряет, насколько ответ соответствует вопросу.

4. **Context_recall** - измеряет способность ретривера извлекать всю необходимую информацию, требуемую для ответа на вопрос.

# Оценка

In [5]:
from ragas import evaluate

result = evaluate(
    my_dataset, 
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)

result

Evaluating:   0%|          | 0/8 [00:00<?, ?it/s]

{'context_precision': 0.5000, 'faithfulness': 0.5833, 'answer_relevancy': 0.9532, 'context_recall': 1.0000}

Если вы хотите углубиться в результаты и выявить примеры, когда ваш трубопровпайплайн работал плохо или исключительно хорошо, вы можете преобразовать его в pandas DataFrame и использовать свои стандартные инструменты аналитики!

In [6]:
df = result.to_pandas()
df

,question,contexts,ground_truth,answer,context_precision,faithfulness,answer_relevancy,context_recall
0,чем представлены органические остатки?,[В протерозойских отложениях органические оста...,известковыми выделениями сине-зелёных водорослей,Органические остатки могут быть представлены р...,0.0,0.166667,0.937413,1.0
1,что найдено в кремнистых сланцах железорудной ...,[В протерозойских отложениях органические оста...,"нитевидные водоросли, грибные нити",В кремнистых сланцах железорудной формации Кан...,1.0,1.000000,0.968983,1.0
